In [1]:
import torchvision as tv
import numpy as np
import torch
import torch.nn.functional as F

In [2]:
transform_train = tv.transforms.Compose(
    [tv.transforms.ToTensor(),
    tv.transforms.RandomHorizontalFlip(),    
    tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform_test = tv.transforms.Compose(
    [tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
train_dataset = tv.datasets.CIFAR10(root="./", train=True,transform=transform_train,download=True)
test_dataset = tv.datasets.CIFAR10(root="./", train=False,transform=transform_test,download=True)
train_loader=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100,shuffle=False)


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [4]:
MODELNAME="cifar_conv.model"
EPOCH = 20
DEVICE="cuda" if torch.cuda.is_available() else "cpu"

In [5]:
class CIFAR_CONV(torch.nn.Module):
  def __init__(self):
    super(CIFAR_CONV,self).__init__()

    # conv1 1/2
    self.conv1_1 = torch.nn.Conv2d(3, 64, kernel_size=3, padding=1)
    self.batchnorm2d1_1 = torch.nn.BatchNorm2d(64)
    self.relu1_1 = torch.nn.ReLU(inplace=True)
    self.conv1_2 = torch.nn.Conv2d(64, 64, kernel_size=3, padding=1)
    self.batchnorm2d1_2 = torch.nn.BatchNorm2d(64)
    self.relu1_2 = torch.nn.ReLU(inplace=True)
    self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    # conv2 1/4
    self.conv2_1 = torch.nn.Conv2d(64, 128, kernel_size=3, padding=1)
    self.batchnorm2d2_1 = torch.nn.BatchNorm2d(128)
    self.relu2_1 = torch.nn.ReLU(inplace=True)
    self.conv2_2 = torch.nn.Conv2d(128, 128, kernel_size=3, padding=1)
    self.batchnorm2d2_2 = torch.nn.BatchNorm2d(128)
    self.relu2_2 = torch.nn.ReLU(inplace=True)
    self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    # conv3 1/8
    self.conv3_1 = torch.nn.Conv2d(128, 256, kernel_size=3, padding=1)
    self.batchnorm2d3_1 = torch.nn.BatchNorm2d(256)
    self.relu3_1 = torch.nn.ReLU(inplace=True)
    self.conv3_2 = torch.nn.Conv2d(256, 256, kernel_size=3, padding=1)
    self.batchnorm2d3_2 = torch.nn.BatchNorm2d(256)
    self.relu3_2 = torch.nn.ReLU(inplace=True)
    self.conv3_3 = torch.nn.Conv2d(256, 256, kernel_size=3, padding=1)
    self.batchnorm2d3_3 = torch.nn.BatchNorm2d(256)
    self.relu3_3 = torch.nn.ReLU(inplace=True)
    self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    # conv4 1/16
    self.conv4_1 = torch.nn.Conv2d(256, 512, kernel_size=3, padding=1)
    self.batchnorm2d4_1 = torch.nn.BatchNorm2d(512)
    self.relu4_1 = torch.nn.ReLU(inplace=True)
    self.conv4_2 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.batchnorm2d4_2 = torch.nn.BatchNorm2d(512)
    self.relu4_2 = torch.nn.ReLU(inplace=True)
    self.conv4_3 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.batchnorm2d4_3 = torch.nn.BatchNorm2d(512)
    self.relu4_3 = torch.nn.ReLU(inplace=True)
    self.pool4 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    # conv5 1/32
    self.conv5_1 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.batchnorm2d5_1 = torch.nn.BatchNorm2d(512)
    self.relu5_1 = torch.nn.ReLU(inplace=True)
    self.conv5_2 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.batchnorm2d5_2 = torch.nn.BatchNorm2d(512)
    self.relu5_2 = torch.nn.ReLU(inplace=True)
    self.conv5_3 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.batchnorm2d5_3 = torch.nn.BatchNorm2d(512)
    self.relu5_3 = torch.nn.ReLU(inplace=True)
    self.pool5 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc1 = torch.nn.Linear(in_features=512 * 1 * 1, out_features=256)
    self.batchnorm1d_fc1 = torch.nn.BatchNorm1d(256)
    self.relu_fc1 = torch.nn.ReLU(inplace=True)
    self.dropout_fc1 = torch.nn.Dropout(0.5)
    self.fc2 = torch.nn.Linear(in_features=256, out_features=80)
    self.batchnorm1d_fc2 = torch.nn.BatchNorm1d(80)
    self.relu_fc2 = torch.nn.ReLU(inplace=True)
    self.dropout_fc2 = torch.nn.Dropout(0.5)
    self.fc3 = torch.nn.Linear(in_features=80, out_features=10)

  def forward(self, x):  
      x = self.relu1_1(self.batchnorm2d1_1(self.conv1_1(x)))
      x = self.relu1_2(self.batchnorm2d1_2(self.conv1_2(x)))  
      x = self.pool1(x) 

      x = self.relu2_1(self.batchnorm2d2_1(self.conv2_1(x)))
      x = self.relu2_2(self.batchnorm2d2_2(self.conv2_2(x)))
      x = self.pool2(x)

      x = self.relu3_1(self.batchnorm2d3_1(self.conv3_1(x)))
      x = self.relu3_2(self.batchnorm2d3_2(self.conv3_2(x)))
      x = self.relu3_3(self.batchnorm2d3_3(self.conv3_3(x)))
      x = self.pool3(x)

      x = self.relu4_1(self.batchnorm2d4_1(self.conv4_1(x)))
      x = self.relu4_2(self.batchnorm2d4_2(self.conv4_2(x)))
      x = self.relu4_3(self.batchnorm2d4_3(self.conv4_3(x)))
      x = self.pool4(x)

      x = self.relu5_1(self.batchnorm2d5_1(self.conv5_1(x)))
      x = self.relu5_2(self.batchnorm2d5_2(self.conv5_2(x)))
      x = self.relu5_3(self.batchnorm2d5_2(self.conv5_3(x)))
      x = self.pool5(x)

      x = x.view(x.size(0), -1)
      x = self.dropout_fc1(self.relu_fc1(self.batchnorm1d_fc1(self.fc1(x))))
      x = self.dropout_fc2(self.relu_fc2(self.batchnorm1d_fc2(self.fc2(x))))
      x = self.fc3(x)

      return x


In [6]:
def train():
  model=CIFAR_CONV().to(DEVICE)
  optimizer= torch.optim.Adam(model.parameters())
  for epoch in range (EPOCH):
    loss=0
    for images, labels in train_loader:
      images=images.view(-1, 3,32,32).to(DEVICE)
      labels=labels.to(DEVICE)
      optimizer.zero_grad()
      y=model(images)
      batchloss=F.cross_entropy(y, labels)
      batchloss.backward()
      optimizer.step()
      loss=loss+batchloss.item()
    print("epoch: {},loss:{}".format(epoch,loss))
    #scheduler.step()
  torch.save(model.state_dict(),MODELNAME)


In [7]:
def test():
  total=len(test_loader.dataset)
  correct=0
  model=CIFAR_CONV().to(DEVICE)
  model.load_state_dict(torch.load(MODELNAME))
  model.eval()

  for images,labels in test_loader:
    images=images.view(-1,3,32,32).to(DEVICE)
    labels=labels.to(DEVICE)
    y=model(images)
    y_pred=y.max(dim=1)[1]
    correct=correct+(y_pred==labels).sum()
  print("correct:{}".format(correct))
  print("total: {}".format(total))
  print("accuracy:{}".format(correct/total))



In [8]:
train()
test()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: 0,loss:894.7800167798996
epoch: 1,loss:634.7507900595665
epoch: 2,loss:490.73005175590515
epoch: 3,loss:412.2823715209961
epoch: 4,loss:358.8572393655777
epoch: 5,loss:324.32452592253685
epoch: 6,loss:285.6037500798702
epoch: 7,loss:259.9010070413351
epoch: 8,loss:237.98952400684357
epoch: 9,loss:210.90393728017807
epoch: 10,loss:197.6508514881134
epoch: 11,loss:182.56094840168953
epoch: 12,loss:163.75888554751873
epoch: 13,loss:148.35144211351871
epoch: 14,loss:135.77693709731102
epoch: 15,loss:129.90488170087337
epoch: 16,loss:119.36628090962768
epoch: 17,loss:108.33292569220066
epoch: 18,loss:99.87782793492079
epoch: 19,loss:95.47039249911904
correct:8712
total: 10000
accuracy:0.8711999654769897
